In [8]:
import os
import glob
import random
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [9]:
imagePath = os.getcwd() + '/uploads/'
histogramPath = os.getcwd() + '/static/histograms/'

In [12]:
def image_processing(file):
    img = cv2.imread(imagePath + str(file), cv2.IMREAD_GRAYSCALE)

    fig = plt.hist(img.ravel(), 256, [0, 256])
    plt.savefig(histogramPath + "histogram1")
    random_value = str(random.randrange(10, 100, 1))
    cancer = random.randrange(10.0, 100.0, 1)
    cv2.imwrite(histogramPath + "before.jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    denoised = cv2.GaussianBlur(img, (5, 5), 0)

    ret, thresh2 = cv2.threshold(denoised, 127, 255, cv2.THRESH_BINARY_INV)

    denoisedTresh = cv2.GaussianBlur(thresh2, (5, 5), 0)

    flood_fill = denoisedTresh.copy()
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = denoisedTresh.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(flood_fill, mask, (0, 0), 255);

    # Invert floodfilled image
    flood_fill_inv = cv2.bitwise_not(flood_fill)

    # Combine the two images to get the foreground.
    filled_out = denoisedTresh | flood_fill_inv

    filter2 = cv2.Laplacian(filled_out, cv2.CV_64F)

    cv2.imwrite(histogramPath + "floodFill" + str(random_value) + ".jpg", flood_fill, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "6Out" + str(random_value) + ".jpg", filled_out, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    cv2.imwrite(histogramPath + "0Started" + str(random_value) + ".jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "1denoised" + str(random_value) + ".jpg", denoised, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "3denoisedThresh" + str(random_value) + ".jpg", denoisedTresh, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "2Tresh" + str(random_value) + ".jpg", thresh2, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "4Mask" + str(random_value) + ".jpg", mask, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    cv2.imwrite(histogramPath + "5invOut" + str(random_value) + ".jpg", flood_fill_inv, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    cv2.imwrite(histogramPath + "7detectedEdges" + str(random_value) + ".jpg", filter2, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
   
    print(filter2)
    
    return cancer

In [13]:
cancer = image_processing("melanomaCancer.jpg")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
